In [15]:
import ads
import pandas as pd
ads.config.token="" 

In [16]:
papers = ads.SearchQuery(q='(((full:"tardis" AND (full:"kerzendorf" OR (bibstem:"Natur" AND full:"supernova")))) AND year:2014-)+property:refereed', sort="date",
                         fl = ['bibcode','title', 'bibstem', 'author', 'year'])

bibcodes = []
titles = []
bibstems = []
authors = []
year_list = []
for paper in papers:
    bibcodes.append(paper.bibcode)
    titles.append(paper.title)
    bibstems.append(paper.bibstem)
    authors.append(paper.author)
    year_list.append(paper.year)

In [17]:
def get_url_link(bibcodes):
    """This function takes the list of bibcodes and returns a link!"""
    url_list = []
    for bibcode in bibcodes:
        url = "https://ui.adsabs.harvard.edu/abs/{}".format(bibcode)
        url_list.append(url)
    return url_list

In [18]:
def get_journal(bibstems):
    """This function takes the list of bibstems and returns the journal that they are in, formatted."""
    journals = []
    for item in bibstems:
        journals.append(item[0])
    return journals

In [55]:
def get_authors_formatted(authors):
    """This gets the first 3 authors of each paper. If there are more than 3, the first 3 followed by 'et al.
    is returned. If it's less than or equal to 3, the first 3 are returned. This function takes in a list and 
    returns a modified version of the list"""
    formatted_author_list=[]
    for author_array in authors: #Note that array is just being used as a variable, no array is used
        count=0
        author_string = ""
        for item in author_array:
            if count==0:
                author_string+=item
                count+=1
            elif count<3:
                author_string+=", "+item
                count+=1
            elif count==3:
                author_string+=", et al."
                break
        formatted_author_list.append(author_string)
    return formatted_author_list

In [56]:
def get_titles_formatted(titles):
    formatted_titles_list = []
    for title in titles:
        title_str = ""
        for item in title:
            title_str += str(item)
        formatted_titles_list.append(title_str)
    return formatted_titles_list

In [57]:
d = {'Authors': get_authors_formatted(authors), 'Year': year_list, 'Journal': get_journal(bibstems), 
     'Title': get_titles_formatted(titles), 'Link': get_url_link(bibcodes)}
df = pd.DataFrame(data=d)
df.to_csv('adslist.csv')

In [58]:
string_list=''

for i in range(len(list(year_list))):
    string_list+=str(get_authors_formatted(authors)[i])+' '+str(year_list[i])+', '+str(get_journal(bibstems)[i])+', "'+str(get_titles_formatted(titles)[i])+'" '+str(get_url_link(bibcodes)[i])+"""
    
    
    """

In [59]:
print(string_list)

Chen, Xingzhuo, Hu, Lei, Wang, Lifan 2020, ApJS, "Artificial Intelligence-Assisted Inversion (AIAI) of Synthetic Type Ia Supernova Spectra" https://ui.adsabs.harvard.edu/abs/2020ApJS..250...12C
    
    
    Miller, A. A., Magee, M. R., Polin, A., et al. 2020, ApJ, "The Spectacular Ultraviolet Flash from the Peculiar Type Ia Supernova 2019yvq" https://ui.adsabs.harvard.edu/abs/2020ApJ...898...56M
    
    
    Gillanders, J. H., Sim, S. A., Smartt, S. J. 2020, MNRAS, "AT2018kzr: the merger of an oxygen-neon white dwarf and a neutron star or black hole" https://ui.adsabs.harvard.edu/abs/2020MNRAS.497..246G
    
    
    Bouquin, Daina R., Chivvis, Daniel A., Henneken, Edwin, et al. 2020, ApJS, "Credit Lost: Two Decades of Software Citation in Astronomy" https://ui.adsabs.harvard.edu/abs/2020ApJS..249....8B
    
    
    Tomasella, Lina, Stritzinger, Maximilian, Benetti, Stefano, et al. 2020, MNRAS, "Observations of the low-luminosity Type Iax supernova 2019gsc: a fainter clone of SN 200

In [60]:
df

,Authors,Year,Journal,Title,Link
0,"Chen, Xingzhuo, Hu, Lei, Wang, Lifan",2020,ApJS,Artificial Intelligence-Assisted Inversion (AI...,https://ui.adsabs.harvard.edu/abs/2020ApJS..25...
1,"Miller, A. A., Magee, M. R., Polin, A., et al.",2020,ApJ,The Spectacular Ultraviolet Flash from the Pec...,https://ui.adsabs.harvard.edu/abs/2020ApJ...89...
2,"Gillanders, J. H., Sim, S. A., Smartt, S. J.",2020,MNRAS,AT2018kzr: the merger of an oxygen-neon white ...,https://ui.adsabs.harvard.edu/abs/2020MNRAS.49...
3,"Bouquin, Daina R., Chivvis, Daniel A., Henneke...",2020,ApJS,Credit Lost: Two Decades of Software Citation ...,https://ui.adsabs.harvard.edu/abs/2020ApJS..24...
4,"Tomasella, Lina, Stritzinger, Maximilian, Bene...",2020,MNRAS,Observations of the low-luminosity Type Iax su...,https://ui.adsabs.harvard.edu/abs/2020MNRAS.49...
5,"Livneh, Ran, Katz, Boaz",2020,MNRAS,An asymmetric explosion mechanism may explain ...,https://ui.adsabs.harvard.edu/abs/2020MNRAS.49...
6,"Kawabata, Miho, Maeda, Keiichi, Yamanaka, Masa...",2020,ApJ,SN 2019ein: New Insights into the Similarities...,https://ui.adsabs.harvard.edu/abs/2020ApJ...89...
7,"Srivastav, Shubham, Smartt, Stephen J., Leloud...",2020,ApJL,The Lowest of the Low: Discovery of SN 2019gsc...,https://ui.adsabs.harvard.edu/abs/2020ApJ...89...
8,"Magee, M. R., Maguire, K., Kotak, R., et al.",2020,A&A,Determining the <SUP>56</SUP>Ni distribution o...,https://ui.adsabs.harvard.edu/abs/2020A&A...63...
9,"Vogl, C., Kerzendorf, W. E., Sim, S. A., et al.",2020,A&A,Spectral modeling of type II supernovae. II. A...,https://ui.adsabs.harvard.edu/abs/2020A&A...63...
